In [ ]:
### cnn keras planesnet

In [33]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.image import *
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os

In [53]:
planes = os.listdir('C:\\Users\\student\\Desktop\\dataset\\python\\planesnet')

In [56]:
categories = []
for name in planes:
    category = name.split('__')[0]
    if category == '0':
        categories.append('X')
    else:
        categories.append('O')
        
df = pd.DataFrame({'filename': planes, 'label':categories})

In [57]:
df.head(3)

,filename,label
0,0__20140723_181317_0905__-122.073653222_37.709...,X
1,0__20140723_181317_0905__-122.083122783_37.722...,X
2,0__20140723_181317_0905__-122.105473207_37.672...,X


In [58]:
df.label.value_counts()

X    24000
O     8000
Name: label, dtype: int64

In [59]:
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

In [69]:
# generator
idg = ImageDataGenerator(rescale=1./255)
train_idg = idg.flow_from_dataframe(train_df, 'C:\\Users\\student\\Desktop\\dataset\\python\\planesnet',
                                              x_col='filename', y_col='label', class_mode='binary', 
                                              target_size=(20,20), batch_size=200)
test_idg = idg.flow_from_dataframe(test_df, 'C:\\Users\\student\\Desktop\\dataset\\python\\planesnet',
                                              x_col='filename', y_col='label', class_mode='binary', 
                                              target_size=(20,20), batch_size=200)

Found 24000 validated image filenames belonging to 2 classes.
Found 8000 validated image filenames belonging to 2 classes.


In [73]:
model = Sequential()
model.add(Conv2D(32, (3,3),input_shape=(20,20,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [74]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [75]:
model.fit_generator(train_idg, epochs=5, steps_per_epoch=110, validation_data=test_idg, validation_steps=40)

Epoch 1/5
110/110 [==============================] - 428s 4s/step - loss: 0.5351 - accuracy: 0.7594 - val_loss: 0.3374 - val_accuracy: 0.8799
Epoch 2/5
110/110 [==============================] - 13s 117ms/step - loss: 0.2978 - accuracy: 0.8758 - val_loss: 0.1894 - val_accuracy: 0.9239
Epoch 3/5
110/110 [==============================] - 12s 114ms/step - loss: 0.2085 - accuracy: 0.9169 - val_loss: 0.1958 - val_accuracy: 0.9463
Epoch 4/5
110/110 [==============================] - 47s 429ms/step - loss: 0.1618 - accuracy: 0.9385 - val_loss: 0.1620 - val_accuracy: 0.9520
Epoch 5/5
110/110 [==============================] - 267s 2s/step - loss: 0.1611 - accuracy: 0.9386 - val_loss: 0.0773 - val_accuracy: 0.9549


In [76]:
model.evaluate_generator(test_idg)[1]

0.9548749923706055